In [9]:
import torch
from torch import nn
from model_conv1d import Encoder, PPFFoldNet
from torchinfo import summary

In [10]:
model = PPFFoldNet()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [32, 64, 1024]             320
       BatchNorm1d-2             [32, 64, 1024]             128
              ReLU-3             [32, 64, 1024]               0
            Conv1d-4            [32, 128, 1024]           8,320
       BatchNorm1d-5            [32, 128, 1024]             256
              ReLU-6            [32, 128, 1024]               0
            Conv1d-7            [32, 256, 1024]          33,024
       BatchNorm1d-8            [32, 256, 1024]             512
              ReLU-9            [32, 256, 1024]               0
           Linear-10           [32, 1024, 1024]         721,920
           Linear-11            [32, 1024, 512]         524,800
          Encoder-12               [32, 1, 512]               0
           Conv1d-13            [32, 256, 1024]         131,840
             ReLU-14            [32, 25

In [11]:
model.to("cuda")

PPFFoldNet(
  (encoder): Encoder(
    (conv1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU()
    (fc1): Linear(in_features=704, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (mlp1): Sequential(
      (0): Conv1d(514, 256, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU()
      (4): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
      (5): ReLU()
      (6): Conv1d(64, 32, kernel_size=(1,)

In [12]:
weights = torch.load("sun3d_best.pkl")
model.load_state_dict(weights)

<All keys matched successfully>

In [13]:
from learning3d.data_utils import ClassificationData, ModelNet40Data
from torch.utils.data import DataLoader
test_dataset = ClassificationData(data_class=ModelNet40Data(train=False, num_points=4096, use_normals=True))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

In [14]:
class PointCloudNormalizer(nn.Module):
    def __init__(self):
        super(PointCloudNormalizer, self).__init__()

    def forward(self, x):
        """
        Input: x -> (B, N, 6)  -> (x, y, z, nx, ny, nz)
        Output: out -> (B, N, 4) -> (x, y, z, norm_magnitude)
        """
        xyz = x[:, :, :3]  # (B, N, 3) - x, y, z koordinatları
        normals = x[:, :, 3:]  # (B, N, 3) - normal vektörleri
        
        # Normallerin büyüklüğünü hesapla (L2 normu)
        norm_magnitude = torch.norm(normals, dim=2, keepdim=True)  # (B, N, 1)

        # Yeni tensörü oluştur (x, y, z, norm büyüklüğü)
        out = torch.cat((xyz, norm_magnitude), dim=2)  # (B, N, 4)
        return out

normalizer_model = PointCloudNormalizer

In [15]:
class PointCloudNormalizer(nn.Module):
    def __init__(self):
        super(PointCloudNormalizer, self).__init__()

    def forward(self, x):
        """
        Input: x -> (B, N, 6)  -> (x, y, z, nx, ny, nz)
        Output: out -> (B, N, 4) -> (x, y, z, norm_magnitude)
        """
        xyz = x[:, :, :3]  # (B, N, 3) - x, y, z koordinatları
        normals = x[:, :, 3:]  # (B, N, 3) - normal vektörleri
        
        # Normallerin büyüklüğünü hesapla (L2 normu)
        norm_magnitude = torch.norm(normals, dim=2, keepdim=True)  # (B, N, 1)

        # Yeni tensörü oluştur (x, y, z, norm büyüklüğü)
        out = torch.cat((xyz, norm_magnitude), dim=2)  # (B, N, 4)
        return out


class EncoderHead(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=256, output_dim=128):
        super(EncoderHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 512 → 256
        self.norm1 = nn.BatchNorm1d(hidden_dim)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # 256 → 128

    def forward(self, x):
        """
        Input: x -> (B, 512)
        Output: out -> (B, 128)
        """
        out = self.fc1(x)
        out = self.norm1(out)
        out = self.act1(out)

        out = self.fc2(out)

        return out


class PPFFoldNetEncoder(nn.Module):
    """Tüm modelleri birleştiren encoder"""
    def __init__(self, input_dim=6, hidden_dim=512, output_dim=128):
        super(PPFFoldNetEncoder, self).__init__()
        self.normalizer = PointCloudNormalizer()
        self.base = model.encoder
        self.head = EncoderHead(512, 256, 128)

    def forward(self, x):
        x = self.normalizer(x)
        x = self.base(x)
        x = self.head(x)
        return x

model = PPFFoldNetEncoder().to("cuda")
torch.save(model.state_dict(), "initial.pkl")  # Sadece ağırlıkları kaydet


In [16]:
model

PPFFoldNetEncoder(
  (normalizer): PointCloudNormalizer()
  (base): Encoder(
    (conv1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU()
    (fc1): Linear(in_features=704, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=512, bias=True)
  )
  (head): EncoderHead(
    (fc1): Linear(in_features=512, out_features=256, bias=True)
    (norm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU()
    (fc2): Linear(in_features=256, out_features=128, bias=True)
  )
)

In [93]:
torch.save(model.state_dict(), "initial.pkl")  # Sadece ağırlıkları kaydet